<a href="https://colab.research.google.com/github/siting1206/NLP_HW2/blob/main/BM25(2%20relative%20snippets).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from gensim import corpora
from gensim.summarization import bm25
from nltk.stem.porter import PorterStemmer  
from collections import namedtuple
import regex as re
import heapq
p_stemmer = PorterStemmer()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
Tag = namedtuple("Tag", ["snippets", "question", "answer"])

# Load train/validation dataset definition
def load_sequences(filename, sep="\t", notypes=False, test_data=False):
    sequences = []
    with open(filename) as fp:
        seq = []
        for line in fp:
            line = line.strip()
            if line:
                line = line.replace(" ||| ", "\t")
                line = line.split(sep)
                seq.append(Tag(*line))
            else:
                sequences.append(seq)
                seq = []
        if seq:
            sequences.append(seq)
    return sequences

In [5]:
train_sequences = load_sequences("drive/MyDrive/NLP_assignment2/Dataset/train.txt", sep="\t", notypes=True)
valid_sequences = load_sequences("drive/MyDrive/NLP_assignment2/Dataset/val.txt", sep="\t", notypes=True)

In [6]:
train_snippets = [[t[0] for t in seq] for seq in (train_sequences)] # length = 99820
train_question = [[t[1] for t in seq] for seq in (train_sequences)]
train_answer = [[t[2] for t in seq] for seq in (train_sequences)]

valid_snippets = [[t[0] for t in seq] for seq in (valid_sequences)] # length = 13893
valid_question = [[t[1] for t in seq] for seq in (valid_sequences)]
valid_answer = [[t[2] for t in seq] for seq in (valid_sequences)]

In [7]:
def sort_index(lst, rev=True):
    index = range(len(lst))
    s = sorted(index, reverse=rev, key=lambda i: lst[i])
    return s

In [8]:
def snippets_filter(snippets, question, answer, trn_s = []):
  
  for i in range(0, len(snippets)):
    snippets[i] = re.findall('<s> (.*?) </s>', snippets[i])
    question[i] = question[i].split()
    answer[i] = answer[i].split()
    snippets_list =[]
    
    for a in snippets[i]:
      a_split = a.replace('?',' ').replace('(',' ').replace(')',' ').split(' ')
      # 詞干提取(e.g. assignment->assign, words->word)
      stemmed_tokens = [p_stemmer.stem(i) for i in a_split] 
      snippets_list.append(stemmed_tokens)
    query =[question[i] + answer[i]]
    query_stemmed = [p_stemmer.stem(i) for i in query[0]]
    # bm25模型
    bm25Model = bm25.BM25(snippets_list)
    # 計算各個snippets與answer相關性
    average_idf = sum(map(lambda k: float(bm25Model.idf[k]), bm25Model.idf.keys())) / len(bm25Model.idf.keys())
    scores = bm25Model.get_scores(query_stemmed,average_idf)
    max_index = sort_index(scores)[:2]
    s = []
    for m in range(0, len(max_index)):
      n = max_index[m]
      s.append(snippets[i][n])
    trn_s.append(s)
  return trn_s, question

In [9]:
print(train_question[0][0], train_answer[0][1])

last 8 years life , galileo house arrest espousing man 's theory jim thorpe


In [10]:
trn_s, trn_q = snippets_filter(train_snippets[0], train_question[0], train_answer[0], trn_s = [])
val_s, val_q = snippets_filter(valid_snippets[0], valid_question[0], valid_answer[0], trn_s = [])
print("training length: ", len(trn_s), len(trn_q))
print("validation length: ", len(val_s), len(val_q))

training length:  99820 99820
validation length:  13893 13893


In [ ]:
for i in range(0, len(trn_s)):
  for j in range(0, len(train_answer[0][i])):
    print(train_answer[0][i][j])

In [33]:
f = open("/content/drive/MyDrive/NLP_assignment2/Dataset/train_ret2.txt", "w")
f.write("snippet\tquestion\tanswer\n")
for i in range(0, len(trn_s)):
  for j in range(0, len(trn_s[i])):
    f.write(trn_s[i][j] + "\t")
    for k in range(0, len(trn_q[i])):
      f.write(trn_q[i][k] + " ")
    f.write("\t")
    for a in range(0, len(train_answer[0][i])):
      f.write(train_answer[0][i][a] + " ")
    f.write("\n")
f.close()

In [34]:
f = open("/content/drive/MyDrive/NLP_assignment2/Dataset/valid_ret2.txt", "w")
f.write("snippet\tquestion\tanswer\n")
for i in range(0, len(val_s)):
  for j in range(0, len(val_s[i])):
    f.write(val_s[i][j] + "\t")
    for k in range(0, len(val_q[i])):
      f.write(val_q[i][k] + " ")
    f.write("\t")
    for a in range(0, len(valid_answer[0][i])):
      f.write(valid_answer[0][i][a] + " ")
    f.write("\n")
f.close()